# 1. Enteros

# La tabla de tipos enteros en NumPy es trivial para cualquier persona con una experiencia mínima en C/C++

# Al igual que en C/C++, 'u' significa 'sin signo' y los dígitos representan el número de bits utilizados para almacenar la variable en la memoria (por ejemplo, np.int64 es un entero con signo de 8 bytes de ancho). Cuando alimenta un int de Python en NumPy, se convierte en un tipo nativo de NumPy llamado np.int32 (o np.int64 según el sistema operativo, la versión de Python y la magnitud de los inicializadores):

In [352]:
 np.array([1, 2, 3]).dtype 


dtype('int32')

# NumPy funciona mejor cuando el ancho de los elementos de la matriz es fijo. Es más rápido y ocupa menos memoria, pero a diferencia de un int ordinario de Python (que funciona en aritmética de precisión arbitraria), los valores de una matriz se ajustarán cuando crucen el valor máximo (o mínimo) para el tipo de datos correspondiente:

In [353]:
np.array([255], np.uint8) [0] + 1




256

In [354]:
np.array([2**31-1])


array([2147483647])

In [355]:
np.array([2**31-1]) + 1


array([-2147483648])

In [356]:
np.array([2**63-1]) + 1


array([-9223372036854775808], dtype=int64)

In [357]:
np.array([255], np.uint8)[0] + 1

256

In [358]:
np.array([2**31-1])[0] + 1

C:\Users\CursosTardes\AppData\Local\Temp\ipykernel_3800\3914130213.py:1: RuntimeWarning: overflow encountered in scalar add
  np.array([2**31-1])[0] + 1


-2147483648

In [359]:
np.array([2**63-1]) [0] + 1

C:\Users\CursosTardes\AppData\Local\Temp\ipykernel_3800\3346125770.py:1: RuntimeWarning: overflow encountered in scalar add
  np.array([2**63-1]) [0] + 1


-9223372036854775808

In [360]:
with np.errstate(over='ignore') :
    print(np.array([2**31-1]) [0]+1)

-2147483648


# Finalmente, si por alguna razón necesita enteros de precisión arbitraria (Python int s) en ndarrays, numpy también es capaz de hacerlo:

In [361]:
a = np.array([10], dtype=object)
len(str(a**1000))

1003

# — pero sin la aceleración habitual, ya que almacenará referencias en lugar de los números en sí, seguirá empaquetando / desempaquetando objetos de Python cuando los procese, etc.

# 2. Flotantes 

# Como el flotante puro de Python no divergió del tipo doble de C estandarizado por IEEE 754 (tenga en cuenta la diferencia en el nombre), la transición de números de coma flotante de Python a NumPy es prácticamente sin problemas: Python float es directamente compatible con np.float64 y Python complex — con np.complex128 .

# Tales advertencias pueden ser 'promovidas' a excepciones o silenciadas a través de errstate o filterwarnings como se describe en la sección 'enteros' anterior, y tal vez para este caso particular eso sería suficiente, pero si realmente desea obtener el valor exacto, puede seleccionar un tipo de archivo más amplio:

In [362]:
x = np.array([-1234.5])
1/(1+np.exp(-x))

C:\Users\CursosTardes\AppData\Local\Temp\ipykernel_3800\2718244023.py:2: RuntimeWarning: overflow encountered in exp
  1/(1+np.exp(-x))


array([0.])

# Una cosa que distingue a los flotantes de los enteros es que son inexactos. No puedes comparar dos flotantes con a == b, a menos que estés seguro de que están representados exactamente. Puede esperar que los flotantes representen exactamente números enteros, pero solo por debajo de un cierto nivel (limitado por el número de dígitos significativos):

In [363]:
np.exp(np.array([1234.5]))


C:\Users\CursosTardes\AppData\Local\Temp\ipykernel_3800\2395929714.py:1: RuntimeWarning: overflow encountered in exp
  np.exp(np.array([1234.5]))


array([inf])

In [364]:
x = np.array([-1234.5]), dtype=np.float128)
1/(1+np.exp(-x))


SyntaxError: unmatched ')' (1839032725.py, line 1)

In [ ]:
9279945539648888.0+1

9279945539648888.0

# También son exactamente representables fracciones como 0.5, 0.125, 0.875 donde el denominador es una potencia de 2 (0.5 = 1/2, 0.125 = 1/8, 0.875 = 7/8, etc.). Cualquier otro denominador dará lugar a un error de redondeo, de modo que 0,1+0,2!=0,3. El enfoque estándar para tratar este problema (así como con la fuente #2 de inexactitud: redondeo de los resultados de los cálculos) es compararlos con una tolerancia relativa (para comparar dos argumentos distintos de cero) y una tolerancia absoluta (si uno de los argumentos es cero). Para los escalares, se maneja mediante math.isclose(a, b, *, rel_tol=1e-09, abs_tol=0.0) , para las matrices NumPy hay una versión vectorial np.isclose(a, b, rtol=1e-05, atol=1e-08) . Tenga en cuenta que las tolerancias tienen diferentes nombres y valores predeterminados. Para los datos financieros decimales. El tipo decimal es útil ya que no implica tolerancias adicionales en absoluto

In [ ]:
from decimal import Decimal as D 
a = np.array([D('0.1'), D('0.2')]); a

array([Decimal('0.1'), Decimal('0.2')], dtype=object)

# Pero no es una bala de plata: también tiene errores de redondeo (ver fuente #2 arriba). El único problema que resuelve es la representación exacta de los números decimales a la que los humanos estamos acostumbrados. Además, no admite nada más complicado que las operaciones aritméticas (aunque se admiten el logaritmo y la raíz cuadrada) y funciona más lento que los flotantes. Para cálculos matemáticos puros, fracciones. Fracción

In [ ]:
a.sum()


Decimal('0.3')

In [ ]:
np.array([1+2j])

array([1.+2.j])

# Al igual que con los números enteros, en matrices flotantes (y complejas) a veces también es útil tratar ciertos valores como 'faltantes'. Los flotantes son más adecuados para almacenar datos anómalos: tienen un valor math.nan (o np.nan o float('nan') ) que se puede almacenar en línea con los valores numéricos 'válidos'. Pero nan es contagiosa en el sentido de que toda la aritmética con nan da como resultado nan. Las funciones estadísticas más comunes tienen una versión resistente a nan (np.nansum, np.nanstd, etc.), pero otras operaciones en esa columna o matriz requerirían un filtrado previo. Las matrices enmascaradas automatizan este paso: la máscara solo se puede construir una vez, luego se "pega" a la matriz original para que todas las operaciones posteriores solo vean los valores desenmascarados y operen sobre ellos.

In [ ]:
a [~np.isnan(a)].mean()

UFuncTypeError: ufunc 'add' cannot use operands with types dtype('<M8[100ms]') and dtype('<M8[100ms]')

In [ ]:
ma.array(a, mask=[0,1,0]).mean()

NameError: name 'ma' is not defined

# 3. Booleanos

# Los valores booleanos se almacenan como bytes individuales para un mejor rendimiento. np.bool_ es un tipo separado del bool de Python porque no necesita conteo de referencias y un enlace a la clase base requerida para cualquier tipo de Python puro. Entonces, si crees que usar 8 bits para almacenar un bit de información es excesivo, mira esto:

In [ ]:
sys.getsizeof(True)

NameError: name 'sys' is not defined

# 4. Cadenas

# Al inicializar una matriz NumPy con una lista de cadenas de Python, se empaquetan en un tipo de dtype nativo de NumPy de ancho fijo llamado np.str_ . Reservar un espacio necesario para que quepa la cadena más larga para cada elemento puede parecer un desperdicio (especialmente en la codificación USC-4 fija en lugar de la elección 'dinámica' del ancho UTF en Python str

In [ ]:
np.array(['abcde','x','y','x'])

array(['abcde', 'x', 'y', 'x'], dtype='<U5')

# La abreviatura '<U4' proviene del llamado protocolo de matriz introducido en 2005. Significa 'cadena codificada en USC-4, 5 elementos de largo' (USC-4≈UTF-32, un ancho fijo, codificación de 4 bytes por carácter). Cada tipo de NumPy tiene una abreviatura, tan ilegible como esta, afortunadamente han adoptado nombres legibles por humanos, al menos para los tipos más utilizados

# Otra opción es mantener las referencias a las entidades de Python en una matriz de objetos NumPy:

In [ ]:
np.array(['abcde','x','y','x'], object)


array(['abcde', 'x', 'y', 'x'], dtype=object)

# Dependiendo de las longitudes relativas de las cadenas y del número de cadenas repetidas, un enfoque puede ser una victoria significativa u otra. Si se trata de una secuencia de bytes sin procesar, NumPy tiene una versión de longitud fija de un tipo de bytes de Python llamado np.bytes_ :

In [ ]:
np.array([b'abcde', b'x', b'y', b'x'])

array([b'abcde', b'x', b'y', b'x'], dtype='|S5')

# Vemos que str_ es más pequeño de nuevo, pero para longitudes más diversas str puede llevarse la victoria. En cuanto a los tipos nativos de np.str_ y np.bytes_, NumPy tiene un puñado de operaciones de cadena comunes. Reflejan los métodos str de Python, viven en el módulo np.char y operan en toda la matriz:

In [ ]:
np.char.upper(np.array([['a','b'],['c','d']]))

array([['A', 'B'],
       ['C', 'D']], dtype='<U1')

# Con las cadenas en modo objeto, los bucles deben ocurrir en el nivel de Python:

In [ ]:
a = np.array([['a','b'],['c','d']], object)
np.vectorize(lambda x: x.upper(), otypes=[object]) (a)

array([['A', 'B'],
       ['C', 'D']], dtype=object)

# Al crear una instancia de np.datetime64 , NumPy elige la granularidad más gruesa que aún puede contener dichos datos:

In [ ]:
np.datetime64('today')

numpy.datetime64('2024-06-18')

In [ ]:
np.datetime64('now')

numpy.datetime64('2024-06-18T16:13:05')

In [ ]:
np.datetime64('2021-12-24 18:14:23.404438123')

numpy.datetime64('2021-12-24T18:14:23.404438123')

# este formato exacto o variaciones mínimas del mismo (ver 'principios generales' de la página de Wikipedia). Una guía completa de los tipos de datos de NumPy | por Lev Maximov | Al crear una matriz, usted decide si está de acuerdo con la granularidad que NumPy ha elegido para usted o si insiste en, digamos, nanosegundos o lo que sea, y le dará 2⁶³ momentos equidistantes medidos en las unidades de tiempo correspondientes a cada lado del 1 de enero de 1970.

In [ ]:
import numpy as np
from datetime import datetime as dt

# Obtener la fecha y hora actual
current_time = dt.utcnow()

# Convertir a datetime64 con nanosegundos
datetime64_ns = np.datetime64(current_time, 'ns')

print(datetime64_ns)


2024-06-18T16:41:26.866337000


In [ ]:
a = np.array([dt.utcnow()], dtype='datetime64[100ms]');a

array(['2024-06-18T16:45:38.100'], dtype='datetime64s]')

In [ ]:
a + 1

array(['2024-06-18T16:45:38.200'], dtype='datetime64s]')

# Para obtener una representación legible por máquina de la granularidad sin analizar la cadena dtype:

In [ ]:
a[0].dtype

dtype('<M8s]')

In [ ]:
np.datetime_data(a[0])

('ms', 100)

In [ ]:
from datetime import timedelta

# Crear un objeto timedelta con 3 días, 36353 segundos y 424753 microsegundos
time_delta = timedelta(days=3, seconds=36353, microseconds=424753)

print(time_delta)


3 days, 10:05:53.424753


In [ ]:
z.item().total_seconds()

NameError: name 'z' is not defined

# O si no te importa la parte fraccionaria, simplemente

In [ ]:
np.datetime64('2022-01-01') - np.datetime64(dt.now(), 's')

numpy.timedelta64(-77741464,'s')

In [ ]:
np.datetime64('2021-12-24 18:14:23').item()

datetime.datetime(2021, 12, 24, 18, 14, 23)

In [ ]:
np.datetime64('2021-12-24 18:14:23').item().month

12

In [ ]:
a = np.arange(np.datetime64('2021-01-20'),
              np.datetime64('2021-12-20'),
              np.timedelta64(90, 'D')); a

array(['2021-01-20', '2021-04-20', '2021-07-19', '2021-10-17'],
      dtype='datetime64[D]')

In [ ]:
s = pd.DatetimeIndex(a); s  # or pd.to_datetime(a)

NameError: name 'pd' is not defined

In [ ]:
s.month 

NameError: name 's' is not defined

In [ ]:
import pandas as pd

# Supongo que 'a' es una lista o array de fechas.
a = ['2020-01-01', '2020-01-02', '2020-01-03']  # Ejemplo de datos

# Crear un DatetimeIndex a partir de 'a'
s = pd.DatetimeIndex(a)
print(s)

# Alternativamente, usar pd.to_datetime(a)
s_alt = pd.to_datetime(a)
print(s_alt)


DatetimeIndex(['2020-01-01', '2020-01-02', '2020-01-03'], dtype='datetime64[ns]', freq=None)
DatetimeIndex(['2020-01-01', '2020-01-02', '2020-01-03'], dtype='datetime64[ns]', freq=None)


In [ ]:
import numpy as np

def dt2cal(dt):
    # Crear un array de salida con la forma correcta
    out = np.empty(dt.shape + (7,), dtype="u4")

    # Extraer componentes individuales
    Y, M, D, h, m, s = [dt.astype(f"M8[{x}]") for x in "YMDhms"]

    # Calcular y asignar cada componente al array de salida
    out[..., 0] = Y + 1970  # Gregorian Year
    out[..., 1] = (M - Y) + 1  # month
    out[..., 2] = (D - M) + 1  # Day
    out[..., 3] = (h - D).astype("m8[h]")  # Hour
    out[..., 4] = (m - h).astype("m8[m]")  # Minute
    out[..., 5] = (s - m).astype("m8[s]")  # Second
    out[..., 6] = (dt - s).astype("m8[us]")  # microsecond

    return out



In [ ]:
dt2cal(a)

AttributeError: 'list' object has no attribute 'shape'

In [ ]:
np.array(['2020-03-01', '2022-03-01','2024-03-01'], np.datetime64) - \

SyntaxError: incomplete input (3147033668.py, line 1)

In [ ]:
from astropy.time import Time

In [ ]:
np.datetime64('2262-01-01', 'ns') - np.datetime64('1678-01-01', 'ns')

numpy.timedelta64(-17537673709551616,'ns')

In [ ]:
np.datetime_as_string(a)

TypeError: input must have type NumPy datetime

In [ ]:
np.datetime_as_string(a, timezone='local')

TypeError: input must have type NumPy datetime

In [ ]:
np.datetime_as_string(a, timezone=pytz.timezone('US/Eastern'))


NameError: name 'pytz' is not defined

# Combinaciones de los mismos 

# Una 'matriz estructurada' en NumPy es una matriz con un tipo de objeto personalizado hecho de los tipos descritos anteriormente como los bloques de construcción básicos (similar a struct en C). Un ejemplo típico es un color de píxel RGB: un tipo de 3 bytes de largo (generalmente 4 para la alineación), en el que se puede acceder a los colores por su nombre:

In [ ]:
rgb = np.dtype([('x', np.uint8), ('y', np.uint8), ('z', np.uint8)])
 a = np.zeros(5, z); a

IndentationError: unexpected indent (2867219527.py, line 2)

In [ ]:
a [0]

10

In [ ]:
a [0] ['x']

TypeError: 'int' object is not subscriptable

In [ ]:
a [0] ['x'] = 10
a

TypeError: 'int' object does not support item assignment

In [ ]:
a['z'] = 5
a 

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

# Para poder acceder a los campos como atributos, se puede utilizar un np.recarray:

In [ ]:
b[0].x

NameError: name 'b' is not defined

In [ ]:
b.y=7; b

NameError: name 'b' is not defined

# Aquí funciona como reinterpret_cast en C++, pero efectivamente, recarray se puede crear por sí solo, sin ser una vista de otra cosa. Los tipos de las matrices estructuradas no tienen que ser necesariamente homogéneos e incluso pueden incluir submatrices. Con matrices estructuradas y recarrays puede obtener la 'apariencia' de un DataFrame básico de Pandas: puede direccionar columnas por nombres, hacer algunos cálculos aritméticos y estadísticos con ellos, puede manejar los valores faltantes de manera eficiente, algunas operaciones son más rápidas en NumPy que en Pandas Pero carecen: agrupación (excepto lo que ofrece itertools.groupby) los poderosos pandas Index y MultiIndex (por lo que no hay tablas dinámicas) y otras sutilezas como una clasificación conveniente,  etc. El problema aquí es que, aunque esta sintaxis es conveniente para abordar columnas particulares como un todo, ni las matrices estructuradas ni las rematrices son algo que desee usar en el bucle más interno de un código intensivo en proceso:

In [ ]:
c = np.recarray(buf=a, shape=len(a), dtype=)
[('x', np.float64),('y', np.float64)])

s1 = 0
for r in a:
    s1 += (r[0]**2 + r[1]**2)**-1.5
    
s2 = 0
for r in b:
    s2 += (r['x']**2 +r['y']**2)**-1.5
    
s3 = 0
for r in c:
    s3 += (r.x**2 + r.y**2)**-1.5
    
    S1 = np.sum((a[:,0]**2 + a[:, 1]**2)**-1.5)
    S2 = np.sum((b['x']**2 + b['y']**2)**-1.5)
    S3 = np.sum((c.x**2 + c.y**2)**-1.5)    

SyntaxError: unmatched ')' (3947015530.py, line 2)

# 7. Comprobaciones de tipo

# Una forma de comprobar el tipo de matriz NumPy es ejecutar isinstance contra su elemento:

In [ ]:
a = np.arrray([1, 2, 3])
v = a[0]
isinstance(v, np.int32)

AttributeError: module 'numpy' has no attribute 'arrray'

# Todos los tipos de NumPy están interconectados en un árbol de herencia que se muestra en la parte superior del artículo (blue=clases abstractas, verde=tipos numéricos, yellow=otros), por lo que en lugar de especificar una lista completa de tipos como isinstance(v, [np.int32, np.int64, etc]) puede escribir comprobaciones de tipos más compactas como


In [ ]:
isinstance(v, np.floating) # true for floats except complex

NameError: name 'v' is not defined

In [ ]:
isinstance(v, np.complexfloating) # true for complex floats only

# La desventaja de este método es que solo funciona contra un valor de la matriz, no contra la matriz en sí. Lo cual no es útil cuando la matriz está vacía, por ejemplo. Comprobar el tipo de matriz es más complicado. Para los tipos básicos, el operador == realiza el trabajo para una sola comprobación de tipos:

In [ ]:
a.dtype == np.int32

False

In [ ]:
a.dtype == np.int64

False

# y en el operador para comprobar con un grupo de tipos:

In [ ]:
x.dtype in (np.half, np.single, np.double, np.longdouble)

True

# Pero para tipos más sofisticados como np.str_ o np.datetime64 no lo hacen. La forma recomendada⁴ de comprobar el dtype con los tipos abstractos es

In [ ]:
np.issubdtype(a.dtype, np.integer)

False

In [ ]:
np.issubdtype(a.dtype, np.floating)

False

# Funciona con todos los tipos nativos de NumPy, pero la necesidad de este método parece algo no obvia: ¿qué hay de malo en el viejo isinstance? Obviamente, la complejidad de la estructura de herencia de los dtypes (¡se construyen 'sobre la marcha'!) no les permitió hacerlo de acuerdo con el principio del menor asombro

In [ ]:
pd.api.types.is_float_dtype(a.dtype)

False

In [ ]:
np.typecodes
{'Character': 'c',
 'Integer': 'bhilqp',
 'UnsignedInteger': 'BHILQP',
 'Float': 'efdg',
 'Complex': 'FDG',
 'AllInteger':'bBhHiIlLqQpP',
 'AllFloat': 'efdgFDG',
 'Datetime': 'Mm',
 'All': '?bhilqpBHILQPefdgFDGSUVOMm'}

{'Character': 'c',
 'Integer': 'bhilqp',
 'UnsignedInteger': 'BHILQP',
 'Float': 'efdg',
 'Complex': 'FDG',
 'AllInteger': 'bBhHiIlLqQpP',
 'AllFloat': 'efdgFDG',
 'Datetime': 'Mm',
 'All': '?bhilqpBHILQPefdgFDGSUVOMm'}

# Su aplicación principal es generar matrices con dtypes específicos para fines de prueba, pero también se puede usar para distinguir entre diferentes grupos de dtypes:

In [ ]:
a.dtype.char in np.typecodes['AllInteger']

False

In [ ]:
a.dtype.char in np.typecodes['Datetime']

False

# Tenga en cuenta que usar a.dtype.kind en lugar de a.dtype.char es un error: np.zeros(1, dtype=np.uint8).dtype.kind == 'u' falta en np.typecodes mientras que <...>.char == 'B' aparece allí. Una desventaja de este método es que bools, strings, bytes, objects y voids ('?', 'U', 'S', 'O' y 'V', respectivamente) no tienen claves dedicadas en el dict. Este enfoque parece más hackish pero menos mágico que issubdtype